In [ ]:
pip install transformers

In [ ]:
pip install -U sentence-transformers

In [ ]:
pip install evaluate

In [3]:
from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import TFAutoModelForSequenceClassification, AutoModel
from transformers import AutoTokenizer, pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request
import re
import numpy as np

In [14]:
from scipy import stats

In [22]:
from evaluate import load

In [4]:
from sentence_transformers import SentenceTransformer, util
import sentence_transformers

In [23]:
glue_metric = load('glue', 'stsb')

In [7]:
sts_tr = pd.read_csv("/content/stsb-en-train.csv", names = ["Sent1", "Sent2", "Score"])
sts_tr

,Sent1,Sent2,Score
0,A plane is taking off.,An air plane is taking off.,5.00
1,A man is playing a large flute.,A man is playing a flute.,3.80
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80
3,Three men are playing chess.,Two men are playing chess.,2.60
4,A man is playing the cello.,A man seated is playing the cello.,4.25
...,...,...,...
5744,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia,0.00
5745,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...,0.00
5746,President heading to Bahrain,President Xi: China to continue help to fight ...,0.00
5747,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders,0.00


In [8]:
sent1 = list(sts_tr["Sent1"])
sent2 = list(sts_tr["Sent2"])
score = list(sts_tr["Score"])

In [13]:
tmp_score = np.array(score)
scaled_score = tmp_score/5
scaled_score.shape

(5749,)

#MPNET V1 Embeddings on STS b Train 

In [ ]:
mpnet_v1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v1')

In [9]:
mpnet_v1_sent1_emb = mpnet_v1.encode(sent1)
mpnet_v1_sent2_emb = mpnet_v1.encode(sent2)

In [ ]:
mp_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(mpnet_v1_sent1_emb[i], mpnet_v1_sent2_emb[i])[0].cpu().T
  mp_v1_score.append(tmp)

mp_v1_score = np.array(mp_v1_score)

In [26]:
results = glue_metric.compute(predictions=mp_v1_score, references=scaled_score)
results

{'pearson': 0.8598188187338868, 'spearmanr': 0.8443610889564444}

#MPNET V2 Embeddings on STS b Train 

In [ ]:
mpnet_v2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [28]:
mpnet_v2_sent1_emb = mpnet_v2.encode(sent1)
mpnet_v2_sent2_emb = mpnet_v2.encode(sent2)

In [ ]:
mp_v2_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(mpnet_v2_sent1_emb[i], mpnet_v2_sent2_emb[i])[0].cpu().T
  mp_v2_score.append(tmp)

mp_v2_score = np.array(mp_v2_score)

In [30]:
results = glue_metric.compute(predictions=mp_v2_score, references=scaled_score)
results

{'pearson': 0.8580658406452041, 'spearmanr': 0.8418432891255972}

#All Roberta Large V1 Embeddings on STS b Train 

In [ ]:
roberta_lg_v1 = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

In [32]:
roberta_lg_v1_sent1_emb = roberta_lg_v1.encode(sent1)
roberta_lg_v1_sent2_emb = roberta_lg_v1.encode(sent2)

In [33]:
roberta_lg_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(roberta_lg_v1_sent1_emb[i], roberta_lg_v1_sent2_emb[i])[0].cpu().T
  roberta_lg_v1_score.append(tmp)

roberta_lg_v1_score = np.array(roberta_lg_v1_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [34]:
results = glue_metric.compute(predictions=roberta_lg_v1_score, references=scaled_score)
results

{'pearson': 0.8624423474026786, 'spearmanr': 0.8484643296795868}

#All MINI LM L12 V1 Embeddings on STS b Train 

In [ ]:
all_MiniLM_L12_v1 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v1')

In [36]:
all_MiniLM_L12_v1_sent1_emb = all_MiniLM_L12_v1.encode(sent1)
all_MiniLM_L12_v1_sent2_emb = all_MiniLM_L12_v1.encode(sent2)

In [37]:
all_MiniLM_L12_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(all_MiniLM_L12_v1_sent1_emb[i], all_MiniLM_L12_v1_sent2_emb[i])[0].cpu().T
  all_MiniLM_L12_v1_score.append(tmp)

all_MiniLM_L12_v1_score = np.array(all_MiniLM_L12_v1_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [38]:
results = glue_metric.compute(predictions=all_MiniLM_L12_v1_score, references=scaled_score)
results

{'pearson': 0.844826511908591, 'spearmanr': 0.8224165007582125}

#MiniLM L6 V2

In [ ]:
all_MiniLM_L6_v2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [67]:
all_MiniLM_L6_v2_sent1_emb = all_MiniLM_L6_v2.encode(sent1)
all_MiniLM_L6_v2_sent2_emb = all_MiniLM_L6_v2.encode(sent2)

In [ ]:
all_MiniLM_L6_v2_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(all_MiniLM_L6_v2_sent1_emb[i], all_MiniLM_L6_v2_sent2_emb[i])[0].cpu().T
  all_MiniLM_L6_v2_score.append(tmp)

all_MiniLM_L6_v2_score = np.array(all_MiniLM_L6_v2_score)

In [69]:
results = glue_metric.compute(predictions=all_MiniLM_L6_v2_score, references=scaled_score)
results

{'pearson': 0.8348826546169758, 'spearmanr': 0.8100236161265245}

#All DistilRoberta V1 Embeddings on STS b Train 

In [ ]:
all_distilroberta_v1 = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [40]:
all_distilroberta_v1_sent1_emb = all_distilroberta_v1.encode(sent1)
all_distilroberta_v1_sent2_emb = all_distilroberta_v1.encode(sent2)

In [ ]:
all_distilroberta_v1_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(all_distilroberta_v1_sent1_emb[i], all_distilroberta_v1_sent2_emb[i])[0].cpu().T
  all_distilroberta_v1_score.append(tmp)

all_distilroberta_v1_score = np.array(all_distilroberta_v1_score)

In [42]:
results = glue_metric.compute(predictions=all_distilroberta_v1_score, references=scaled_score)
results

{'pearson': 0.8517514218741723, 'spearmanr': 0.8288352774536412}

#AllenAi Specter on STS b train

In [ ]:
allen_specter = SentenceTransformer('sentence-transformers/allenai-specter')

In [63]:
allen_specter_sent1_emb = allen_specter.encode(sent1)
allen_specter_sent2_emb = allen_specter.encode(sent2)

In [ ]:
allen_specter_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(allen_specter_sent1_emb[i], allen_specter_sent2_emb[i])[0].cpu().T
  allen_specter_score.append(tmp)

allen_specter_score = np.array(allen_specter_score)

In [65]:
results = glue_metric.compute(predictions=allen_specter_score, references=scaled_score)
results

{'pearson': 0.6462878536912912, 'spearmanr': 0.6112146043224389}

#Mini LM Ensemble

In [70]:
v_add_sent1 = all_MiniLM_L12_v1_sent1_emb + all_MiniLM_L6_v2_sent1_emb
v_add_sent2 = all_MiniLM_L12_v1_sent2_emb + all_MiniLM_L6_v2_sent2_emb

In [ ]:
mini_lm_ensemble_score = []
for i in range(len(sent1)):
  tmp = util.cos_sim(v_add_sent1[i], v_add_sent2[i])[0].cpu().T
  mini_lm_ensemble_score.append(tmp)

mini_lm_ensemble_score = np.array(mini_lm_ensemble_score)

In [72]:
results = glue_metric.compute(predictions=mini_lm_ensemble_score, references=scaled_score)
results

{'pearson': 0.8393035392174015, 'spearmanr': 0.8185991793470453}

Horizontal Ensemble

In [74]:
h_add_sent1 = np.concatenate((all_MiniLM_L12_v1_sent1_emb, all_MiniLM_L6_v2_sent1_emb), axis=1)
h_add_sent2 = np.concatenate((all_MiniLM_L12_v1_sent2_emb, all_MiniLM_L6_v2_sent2_emb), axis=1)

In [75]:
h_add_lm_ensemble_score = []

for i in range(len(sent1)):
  tmp = util.cos_sim(h_add_sent1[i], h_add_sent2[i])[0].cpu().T
  h_add_lm_ensemble_score.append(tmp)

h_add_lm_ensemble_score = np.array(h_add_lm_ensemble_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [76]:
results = glue_metric.compute(predictions=h_add_lm_ensemble_score, references=scaled_score)
results

{'pearson': 0.844262754071183, 'spearmanr': 0.8213579942926563}

#Embedding Ensemble MpNET V1- Distill Roberta V1

Vertical Ensemble

In [50]:
v_add_sent1 = mpnet_v1_sent1_emb + all_distilroberta_v1_sent1_emb
v_add_sent2 = mpnet_v1_sent2_emb + all_distilroberta_v1_sent2_emb

In [ ]:
v_add_mpnet_distill_score = []

for i in range(len(sent1)):
  tmp = util.cos_sim(v_add_sent1[i], v_add_sent2[i])[0].cpu().T
  v_add_mpnet_distill_score.append(tmp)

v_add_mpnet_distill_score = np.array(v_add_mpnet_distill_score)

In [52]:
results = glue_metric.compute(predictions=v_add_mpnet_distill_score, references=scaled_score)
results

{'pearson': 0.8615021241736818, 'spearmanr': 0.8438469647590215}

Horizontal Ensemble

In [57]:
h_add_sent1 = np.concatenate((mpnet_v1_sent1_emb, all_distilroberta_v1_sent1_emb), axis=1)
h_add_sent2 = np.concatenate((mpnet_v1_sent2_emb, all_distilroberta_v1_sent2_emb), axis=1)

In [ ]:
h_add_mpnet_distill_score = []

for i in range(len(sent1)):
  tmp = util.cos_sim(h_add_sent1[i], h_add_sent2[i])[0].cpu().T
  h_add_mpnet_distill_score.append(tmp)

h_add_mpnet_distill_score = np.array(h_add_mpnet_distill_score)

In [60]:
results = glue_metric.compute(predictions=h_add_mpnet_distill_score, references=scaled_score)
results

{'pearson': 0.8617699573273537, 'spearmanr': 0.8441417031313329}